In [1]:
import numpy as np

In [4]:
park1_loop_fn = "results_avia_park/std_result_park1.csv"
park2_loop_fn = "results_avia_park/std_result_park2.csv"
park1_time_fn = "results_avia_park/std_consumption_park1.csv"
park2_time_fn = "results_avia_park/std_consumption_park2.csv"

In [5]:
park1_loop = np.genfromtxt(park1_loop_fn, delimiter=',')
park2_loop = np.genfromtxt(park2_loop_fn, delimiter=',')
park1_time = np.genfromtxt(park1_time_fn, delimiter=',')
park2_time = np.genfromtxt(park2_time_fn, delimiter=',')
print("park1_loop.shape: ", park1_loop.shape)
print("park2_loop.shape: ", park2_loop.shape)
print("park1_time.shape: ", park1_time.shape)
print("park2_time.shape: ", park2_time.shape)

park1_loop.shape:  (598, 8)
park2_loop.shape:  (539, 8)
park1_time.shape:  (598, 3)
park2_time.shape:  (539, 3)


In [10]:
park1_loop[-1, 0] - park1_loop[0, 0] 

597.1000339984894

### Avia Park 1 Dataset

In [34]:
SKIP_NUM = 50
SKIP_NEAR_NUM = 100
ARG_SCORE_THRES = 0.1
ARG_LOOP_THRES = 10.0

In [36]:
loop_data = park1_loop
time_data = park1_time

assert(loop_data.shape[0] == time_data.shape[0])
N_DATA = loop_data.shape[0]

if N_DATA <= SKIP_NUM:
    print(f"[WARN] dataset length({N_DATA}) is smaller than skip length({SKIP_NUM})")
    exit(0)

for i in range(SKIP_NUM, N_DATA): #
    t_sec = loop_data[i,0]
    score = loop_data[i,1]
    t_query = loop_data[i,2:5].reshape((3,1))
    t_dbase = loop_data[i,5:8].reshape((3,1))
    
    is_loop = False
    
    loop_candidates = loop_data[0:i-SKIP_NUM+1,2:5]
    loop_candidates_dist:np.ndarray = loop_candidates - np.tile(t_query.reshape((1,3)), (loop_candidates.shape[0],1))
    loop_candidates_dist = np.linalg.norm(loop_candidates_dist, axis=1)
    min_dist = 9999.0
    # print("loop_candidates_dist: ", loop_candidates_dist.shape)
    # print(loop_candidates_dist.tolist())
    if (loop_candidates_dist <= ARG_LOOP_THRES).any():
        # print(f"Loop exists")
        is_loop = True
        min_dist = loop_candidates_dist.min()
    
    if score >= ARG_SCORE_THRES:
        l2_dist = ((t_query - t_dbase) ** 2)
        l2_dist = np.sqrt(np.sum(l2_dist))
        
        if is_loop:
            print(f"[Succ] Idx({i:03d}): Score({score:2.4f} / Dist({l2_dist:2.4f}))")  
        else:
            print(f"[Fail] Idx({i:03d}): Score({score:2.4f} / Dist({l2_dist:2.4f}))")  
    else:
        if is_loop:
            print(f"Fail to detect loop, Score({score:2.4f} vs Min({min_dist:2.4f})")
        else:
            pass

[Fail] Idx(150): Score(0.1083 / Dist(53.7319))
[Fail] Idx(157): Score(0.1650 / Dist(49.2948))
[Fail] Idx(161): Score(0.1013 / Dist(47.5823))
Fail to detect loop, Score(0.0110 vs Min(9.1732)
Fail to detect loop, Score(0.0375 vs Min(7.6480)
Fail to detect loop, Score(0.0000 vs Min(6.1241)
Fail to detect loop, Score(0.0117 vs Min(4.6932)
Fail to detect loop, Score(0.0000 vs Min(3.3254)
Fail to detect loop, Score(0.0000 vs Min(1.8989)
Fail to detect loop, Score(0.0178 vs Min(1.1794)
Fail to detect loop, Score(0.0461 vs Min(0.6271)
Fail to detect loop, Score(0.0000 vs Min(0.6053)
[Succ] Idx(183): Score(0.2908 / Dist(0.9718))
[Succ] Idx(184): Score(0.3529 / Dist(0.8841))
[Succ] Idx(185): Score(0.4468 / Dist(0.7835))
Fail to detect loop, Score(0.0066 vs Min(0.6803)
[Succ] Idx(187): Score(0.5101 / Dist(0.5562))
Fail to detect loop, Score(0.0000 vs Min(0.9795)
Fail to detect loop, Score(0.0000 vs Min(2.3608)
Fail to detect loop, Score(0.0000 vs Min(3.8313)
Fail to detect loop, Score(0.0064 vs M